In [ ]:
#=================================================튜닝=========================================================

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from lightgbm import LGBMClassifier

In [ ]:
# ✅ 1. 데이터 불러오기
df = pd.read_csv('final_cleaned.csv')

In [ ]:
# ✅ 2. file_ext가 없을 경우 URL에서 확장자 추출 (보완형 처리)
if 'file_ext' not in df.columns:
    df['file_ext'] = df['url'].str.extract(r'\.([a-zA-Z0-9]+)$').fillna('none')

In [ ]:
# ✅ 3. URL 기반 파생 피처 생성
df['url_length'] = df['url'].apply(lambda x: len(str(x)))
df = df.drop('url', axis=1)  # 원본 URL 문자열 제거

In [ ]:
# ✅ 4. file_ext 인코딩
le = LabelEncoder()
df['file_ext'] = le.fit_transform(df['file_ext'])

In [ ]:
# ✅ 5. 피처(X), 레이블(y) 분리
X = df.drop('label', axis=1)
y = df['label']

In [ ]:
# ✅ 6. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

In [ ]:
# ✅ 7. 튜닝된 LightGBM 모델 정의
tuned_model = LGBMClassifier(
    num_leaves=64,
    max_depth=10,
    learning_rate=0.05,
    n_estimators=500,
    min_child_samples=30,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42
)

In [ ]:
# ✅ 8. 모델 학습
tuned_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.326019 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1525
[LightGBM] [Info] Number of data points in the train set: 509554, number of used features: 41
[LightGBM] [Info] Start training from score -0.626363
[LightGBM] [Info] Start training from score -1.803029
[LightGBM] [Info] Start training from score -1.201746
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, max_depth=10,
               min_child_samples=30, n_estimators=500, num_leaves=64,
               random_state=42, reg_alpha=0.1, reg_lambda=0.1, subsample=0.8)

In [ ]:
# ✅ 9. 예측
y_pred = tuned_model.predict(X_test)

In [ ]:

# ✅ 10. 평가 출력
print("📊 [Confusion Matrix]")
print(np.array2string(confusion_matrix(y_test, y_pred), separator=' '))

print("\n📈 [Classification Report]")
print(classification_report(y_test, y_pred, digits=4))

acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")

📊 [Confusion Matrix]
[[54108  1026 12959]
 [  430 16824  3740]
 [ 6713  2040 29549]]

📈 [Classification Report]
              precision    recall  f1-score   support

         0.0     0.8834    0.7946    0.8367     68093
         1.0     0.8459    0.8014    0.8230     20994
         2.0     0.6389    0.7715    0.6990     38302

    accuracy                         0.7888    127389
   macro avg     0.7894    0.7892    0.7862    127389
weighted avg     0.8037    0.7888    0.7930    127389


✅ Accuracy: 0.7888
